In [1]:
import pandas as pd

In [2]:
# Принято добавлять к названию переменной _df, чтобы было понятно, что это DataFrame
flights_df = pd.read_csv("D:/Lern ht code/python_flights_project/data/flights.csv")
airlines_df = pd.read_csv("D:/Lern ht code/python_flights_project/data/airlines.csv")
airports_df = pd.read_csv("D:/Lern ht code/python_flights_project/data/airports.csv")
planes_df = pd.read_csv("D:/Lern ht code/python_flights_project/data/planes.csv")
weather_df = pd.read_csv("D:/Lern ht code/python_flights_project/data/weather.csv")

flights_df['time_hour'] = pd.to_datetime(flights_df['time_hour'])
weather_df['time_hour'] = pd.to_datetime(weather_df['time_hour'])
weather_df['time_hour'] = weather_df['time_hour'].dt.tz_localize(None)

'''
Преобразуем колонку time_hour в flights_df и weather_df в тип datetime64[ns], потому что был object
Потому что нельзя было делать merge по дате/времени

У flights_df был тип - datetime64[ns]. У weather_df - datetime64[ns, UTC]. Убрал таймзону UTC
'''

'\nПреобразуем колонку time_hour в flights_df и weather_df в тип datetime64[ns], потому что был object\nПотому что нельзя было делать merge по дате/времени\n\nУ flights_df был тип - datetime64[ns]. У weather_df - datetime64[ns, UTC]. Убрал таймзону UTC\n'

In [ ]:
highest_no = planes_with_flights = planes_df.merge(flights_df, on='tailnum', how='left')
highest_no = highest_no.merge(airlines_df, on='carrier', how='left')


result = (
    highest_no
    .groupby(['tailnum', 'carrier', 'name'])
    .size()                                        # COUNT(*) аналог
    .reset_index(name='flight_count')              # переименовали колонку
    .sort_values('flight_count', ascending=False)  # ORDER BY flight_count DESC
)
print(result)

result = (
    flights_df
    .groupby('tailnum')['carrier']
    .nunique()                                   # аналог COUNT(DISTINCT carrier)
    .reset_index(name='carriers_count')          # колонка вместо индекса
    .query("carriers_count > 1")                 # HAVING
    .sort_values('carriers_count', ascending=False)
)

print(result.head())
'''
-- У какого самолёта самое большое количество рейсов и его компания
SELECT pl.tailnum, fl.carrier, al.name AS airline_name, COUNT(*) AS flight_count
FROM planes AS pl
LEFT JOIN flights AS fl
USING(tailnum)
LEFT JOIN airlines AS al 
USING(carrier)
GROUP BY pl.tailnum, fl.carrier, al.name  -- группируем по всем трём полям
ORDER BY flight_count DESC

/* Самое большое кол-во рейсов у самолёта с tailnum=N711MQ компании Envoy Air, он совершил 18 рейсов */;
SELECT tailnum, COUNT(DISTINCT carrier) as carriers_count
FROM flights
GROUP BY tailnum
HAVING COUNT(DISTINCT carrier) > 1
ORDER BY carriers_count DESC
/* ДО поделения на carrier было 3322 строки, после - 3326
Запрос выше показывает, сколько и какие имменно самолёты летали больше чем за одну компанию
*/;
'''

     tailnum carrier                         name  flight_count
616   N335AA      AA       American Airlines Inc.            18
1599  N711MQ      MQ                    Envoy Air            18
315   N183JB      B6              JetBlue Airways            17
629   N339AA      AA       American Airlines Inc.            17
1420  N632JB      B6              JetBlue Airways            17
...      ...     ...                          ...           ...
2257  N960AT      FL  AirTran Airways Corporation             1
2259  N961AT      FL  AirTran Airways Corporation             1
937   N426AA      AA       American Airlines Inc.             1
2264  N963AT      FL  AirTran Airways Corporation             1
2280  N969AT      FL  AirTran Airways Corporation             1

[2328 rows x 4 columns]
     tailnum  carriers_count
2803  N998AT               1
0     N0EGMQ               1
2     N104UW               1
5     N107US               1
6     N109UW               1


'\n-- У какого самолёта самое большое количество рейсов и его компания\nSELECT pl.tailnum, fl.carrier, al.name AS airline_name, COUNT(*) AS flight_count\nFROM planes AS pl\nLEFT JOIN flights AS fl\nUSING(tailnum)\nLEFT JOIN airlines AS al \nUSING(carrier)\nGROUP BY pl.tailnum, fl.carrier, al.name  -- группируем по всем трём полям\nORDER BY flight_count DESC\n\n/* Самое большое кол-во рейсов у самолёта с tailnum=N711MQ компании Envoy Air, он совершил 18 рейсов */;\nSELECT tailnum, COUNT(DISTINCT carrier) as carriers_count\nFROM flights\nGROUP BY tailnum\nHAVING COUNT(DISTINCT carrier) > 1\nORDER BY carriers_count DESC\n/* ДО поделения на carrier было 3322 строки, после - 3326\nЗапрос выше показывает, сколько и какие имменно самолёты летали больше чем за одну компанию\n*/;\n'

In [12]:
full_join = airlines_df.merge(flights_df, on='carrier', how='outer')

result = (
    full_join
    .loc[full_join['carrier'].isin(['UA', 'AA']),
    ['carrier', 'name', 'month', 'minute', 'dep_time']]
)
print(result)
# isin - находится ли значение в колонке среди заданного списка
'''
-- Простой запрос на FULL JOIN
SELECT carrier, al.name, fl.month, fl.minute, fl.dep_time
FROM airlines as al
FULL JOIN flights as fl
USING(carrier)
WHERE carrier='UA' OR carrier='AA';
'''

     carrier                    name  month  minute  dep_time
521       AA  American Airlines Inc.      5      55    1951.0
522       AA  American Airlines Inc.      7      45    1336.0
523       AA  American Airlines Inc.      4       0    1458.0
524       AA  American Airlines Inc.      3      30     847.0
525       AA  American Airlines Inc.      6      15    1311.0
...      ...                     ...    ...     ...       ...
8847      UA   United Air Lines Inc.      9      30     628.0
8848      UA   United Air Lines Inc.      1       0    1949.0
8849      UA   United Air Lines Inc.      4      45     841.0
8850      UA   United Air Lines Inc.     12       5    1502.0
8851      UA   United Air Lines Inc.     12      56    1803.0

[2699 rows x 5 columns]


"\n-- Простой запрос на FULL JOIN\nSELECT carrier, al.name, fl.month, fl.minute, fl.dep_time\nFROM airlines as al\nFULL JOIN flights as fl\nUSING(carrier)\nWHERE carrier='UA' OR carrier='AA';\n"

In [22]:
cross_join = flights_df.merge(airlines_df, how='cross', suffixes=("_flights", "_airlines"))

result = (
    cross_join
    .loc[(cross_join['origin']=='JFK') & (cross_join['carrier_airlines']=='HA'),
         ['dest', 'origin', 'name']]
)
print(result)


# каждая строка из flights_df соединяется с каждой строкой из airlines_df.
# carrier_flights == "HA" ->> ТОЛЬКО те рейсы, где перевозчик в FLIGHTS_DF = Hawaiian Airlines.
# carrier_y == "HA" ->> ВСЕ рейсы, но остаются только те строки, где справочник AIRLINES_DF УКАЗЫВАЕТ НА HAWAIIAN.

'''
-- Простой запрос на CROSS JOIN
SELECT fl.dest, fl.origin, al.name
FROM flights AS fl
CROSS JOIN airlines AS al
WHERE fl.origin IN ('JFK') AND al.carrier IN ('HA');
'''

       dest origin                    name
72      TPA    JFK  Hawaiian Airlines Inc.
88      BOS    JFK  Hawaiian Airlines Inc.
104     SFO    JFK  Hawaiian Airlines Inc.
120     IAD    JFK  Hawaiian Airlines Inc.
168     CHS    JFK  Hawaiian Airlines Inc.
...     ...    ...                     ...
159880  IND    JFK  Hawaiian Airlines Inc.
159896  SAN    JFK  Hawaiian Airlines Inc.
159912  SEA    JFK  Hawaiian Airlines Inc.
159928  LAX    JFK  Hawaiian Airlines Inc.
159960  PBI    JFK  Hawaiian Airlines Inc.

[3276 rows x 3 columns]


"\n-- Простой запрос на CROSS JOIN\nSELECT fl.dest, fl.origin, al.name\nFROM flights AS fl\nCROSS JOIN airlines AS al\nWHERE fl.origin IN ('JFK') AND al.carrier IN ('HA');\n"

In [ ]:
self_join = flights_df.merge(flights_df, on=(['month','day']))

result = (
    flights_df
    .merge(flights_df, on="month", suffixes=("_fl1", "_fl2"))
    .query("day_fl1 != day_fl2")
    [["day_fl1", "day_fl2", "month"]]
)
print(result.head(40))

'''
-- SELF JOIN (сравнивать колонку с разными значениями в ней самой, в одной таблице)

-- Сейчас написал сравнение дней в колонке по месяцам из flights
SELECT fl1.day AS day1, fl2.day AS day2, fl1.month
FROM flights AS fl1
INNER JOIN flights AS fl2
ON fl1.month=fl2.month
AND fl1.day<>fl2.day
LIMIT 40
/* Запрос (условно): 
Берёт сначала «1-й день января со всеми остальными днями января»,
потом «2-й день января со всеми остальными», и так далее
*/;
'''

    day_fl1  day_fl2  month
1        27        9      6
2        27        4      6
3        27       17      6
4        27       14      6
5        27       24      6
6        27        7      6
7        27       28      6
8        27       26      6
9        27       19      6
10       27       22      6
11       27        2      6
12       27       21      6
13       27       30      6
14       27       13      6
15       27       20      6
16       27       16      6
17       27       18      6
18       27       17      6
19       27        3      6
20       27       12      6
21       27       23      6
22       27       23      6
23       27       22      6
24       27       28      6
26       27        3      6
27       27       24      6
28       27        1      6
29       27        3      6
30       27        1      6
31       27       20      6
32       27       28      6
33       27        7      6
34       27       30      6
35       27       20      6
36       27        4

'\n-- SELF JOIN (сравнивать колонку с разными значениями в ней самой, в одной таблице)\n\n-- Сейчас написал сравнение дней в колонке по месяцам из flights\nSELECT fl1.day AS day1, fl2.day AS day2, fl1.month\nFROM flights AS fl1\nINNER JOIN flights AS fl2\nON fl1.month=fl2.month\nAND fl1.day<>fl2.day\nLIMIT 40\n/* Запрос (условно): \nБерёт первый рейс 1-го января из таблицы fl1.\nСоединяет его со всеми сотнями рейсов, которые произошли 2-го января в таблице fl2.\nПотом соединяет его со всеми сотнями рейсов 3-го января и т.д.\nЗатем он берёт второй рейс 1-го января и повторяет весь процесс.\n*/;\n'

In [38]:
same_time = flights_df.merge(flights_df, on=["dest", "day", "month"], suffixes=("_fl1", "_fl2"))

result = (
    same_time
    .loc[
         (same_time["sched_arr_time_fl1"] > same_time["sched_arr_time_fl2"]) &
         ((same_time["sched_arr_time_fl1"] - same_time["sched_arr_time_fl2"]) <= 5),
         ["flight_fl1", "flight_fl2", "origin_fl1", "origin_fl2", "dest", "sched_arr_time_fl1", "sched_arr_time_fl2"]
    ]
    .assign(difference=lambda df: df["sched_arr_time_fl1"] - df["sched_arr_time_fl2"])
    .sort_values("difference", ascending=True)
)
print(result)

'''
SELECT fl1.flight AS first_flight, fl2.flight AS second_flight, fl1.origin AS first_origin, fl2.origin AS second_origin, fl1.dest, fl2.sched_arr_time AS second_arr_time, fl1.sched_arr_time AS first_arr_time, fl1.sched_arr_time - fl2.sched_arr_time AS difference
FROM flights AS fl1
INNER JOIN flights AS fl2
ON fl1.dest = fl2.dest
AND fl1.day = fl2.day
AND fl1.month = fl2.month
AND fl1.sched_arr_time > fl2.sched_arr_time
AND (fl1.sched_arr_time - fl2.sched_arr_time) <= 5
ORDER BY difference ASC
/* Данный SQL-запрос выполняет поиск пар рейсов с одинаковым пунктом назначения (dest), 
которые прибывают в один и тот же день с интервалом не более 5 минут */;
'''

       flight_fl1  flight_fl2 origin_fl1 origin_fl2 dest  sched_arr_time_fl1  \
1051          127         695        EWR        JFK  MCO                1350   
1313          409         219        EWR        JFK  CLT                1023   
1767         4440           8        EWR        JFK  BUF                1556   
3419          763         399        JFK        JFK  LAX                1011   
5122          745        3676        EWR        LGA  CLT                 826   
4958         1047        2357        EWR        JFK  SAN                2158   
11314         177        1495        JFK        EWR  SFO                2210   
2701          935        2247        EWR        LGA  ATL                1631   
12652        5207        2095        LGA        EWR  CLT                1507   
12405         348         781        LGA        LGA  ATL                1535   
11539         501         371        JFK        LGA  FLL                 854   
4583          245         561        EWR

'\nSELECT fl1.flight AS first_flight, fl2.flight AS second_flight, fl1.origin AS first_origin, fl2.origin AS second_origin, fl1.dest, fl2.sched_arr_time AS second_arr_time, fl1.sched_arr_time AS first_arr_time, fl1.sched_arr_time - fl2.sched_arr_time AS difference\nFROM flights AS fl1\nINNER JOIN flights AS fl2\nON fl1.dest = fl2.dest\nAND fl1.day = fl2.day\nAND fl1.month = fl2.month\nAND fl1.sched_arr_time > fl2.sched_arr_time\nAND (fl1.sched_arr_time - fl2.sched_arr_time) <= 5\nORDER BY difference ASC\n/* Данный SQL-запрос выполняет поиск пар рейсов с одинаковым пунктом назначения (dest), \nкоторые прибывают в один и тот же день с интервалом не более 5 минут */;\n'